In [1]:
#import zipfile
import numpy as np
import pandas
import os
import io
from pprint import pprint
import json
from datetime import datetime
print('imported')

imported


In [2]:
# import list of teams from retrosheet
teams_df = pandas.DataFrame.from_csv('teams.csv')
teams_df['firstGame'] = pandas.to_datetime(teams_df['firstGame'], errors='coerce')
teams_df['lastGame'] = pandas.to_datetime(teams_df['lastGame'], errors='coerce')

teams_df['firstYear'] = pandas.to_numeric(teams_df['firstGame'].apply(lambda x: x.year), downcast='integer', errors='coerce')
teams_df['lastYear'] = pandas.to_numeric(teams_df['lastGame'].apply(lambda x: x.year), downcast='integer', errors='coerce')
teams_df['lastYear'] = teams_df['lastYear'].fillna(2016)

teams_df = teams_df.fillna('')
print('ok')
#teams_df

ok


In [ ]:
allfiles = []
allgames = []
gamecount = -1  # MUST START AT -1

ladj_games = []
badj_games = []
padj_games = []

yeararray = [1921, 1922, 1925, 1927, 1928]
for i in np.linspace(1930,2016,87):
    yeararray.append(i)

for thisyear in yeararray:
    print('Starting to parse the year {0:.0f}...'.format(thisyear))
    allfiles = []
    allgames = []
    gamecount = -1  # MUST START AT -1
    teams_this_year_df = teams_df[(teams_df['firstYear'] <= thisyear) & (teams_df['lastYear'] >= thisyear)]
    for oneindex, oneteam in teams_this_year_df.iterrows():
        oneteamname = oneteam[0]
        itsleague = oneteam[1][0:1]
        afile = '{:.0f}'.format(thisyear)+oneteamname+'.EV'+itsleague
        if (afile in (os.listdir('retrosheet/'))):
            allfiles.append('retrosheet/'+afile)
#    print('files identified')
    
    for filename in allfiles:
#        print('Parsing file: ',filename)
        for line in io.open(filename,encoding='utf-8').readlines():
            linearray = line.split(',')
            linearray[-1] = linearray[-1][0:-1]  # cut off newline character
            if (linearray[0] == 'id'):
                # 'id' starts the next game, so immediately advance gamecount  
                # record prior game data by adding it to the gamelogs list, unless gamecount = 0 in which case no data yet
                if (gamecount >= 0):
                    thisgamedict['info'] = info_dict
                    thisgamedict['visiting_starters'] = vis_start_list
                    thisgamedict['home_starters'] = home_start_list
                    thisgamedict['events'] = events
                    thisgamedict['data'] = data_list
                    # append the dictionary describing this game to the list of games
                    allgames.append(thisgamedict)

                # advance the gamecount and initialize a new game
                gamecount = gamecount + 1
#                print('Parsing game number {0:,.0f} (0-based counting)'.format(gamecount))
                thisgamedict = {}
                info_dict = {}
                start_dict = {}
                vis_start_list = []
                home_start_list = []
                vis_sub_list = []
                home_sub_list = []
                events = []
#                events = np.empty((0,12),dtype=object)
                data_list = []
                eventnum = 1

                # the first thing to go into the new game is the game id.
                # the [0:-1] is there to cut off the newline character
                thisgamedict['id'] = linearray[1]

            elif (linearray[0] == 'version'):
                thisgamedict['version'] = linearray[1]
            elif (linearray[0] == 'info'):
                info_dict[linearray[1]] = linearray[2]
            elif (linearray[0] == 'start'):
                thisbatter = {}
                thisbatter['batter_id'] = linearray[1]
                thisbatter['batter_name'] = linearray[2]
                thisbatter['order'] = linearray[4]
                thisbatter['position'] = linearray[5]

                if (linearray[3] == '0'):
                    vis_start_list.append(thisbatter)
                elif (linearray[3] == '1'):
                    home_start_list.append(thisbatter)
                else:
                    print('Neither visitor nor home!!!!')

            elif (linearray[0] == 'data'):
#                print(linearray)
                thisdata = {}
                thisdata['type'] = linearray[1]
                thisdata['player'] = linearray[2]
                thisdata['thevalue'] = linearray[3]
                data_list.append(thisdata)
            else:
                thisevent = {}
                thisevent['event_in_game'] = eventnum
                thisevent['event_type'] = linearray[0]
                
                if (thisevent['event_type'] == 'com'):
                    thisevent['comment'] = linearray[1]
                    
                elif (thisevent['event_type'] == 'sub'):
                    thisevent['sub_id'] = linearray[1]
                    thisevent['sub_team'] = linearray[3]
                    thisevent['sub_name'] = linearray[2]
                    thisevent['sub_batting_order'] = linearray[4]
                    thisevent['sub_position'] = linearray[5]

                    if (thisevent['sub_team'] not in ('0','1')):
#                        print(linearray)
                        if ((thisyear == 1932) & (linearray[1] == 'watkg101')):
                            thisevent['sub_team'] = linearray[4]
                            thisevent['sub_batting_order'] = linearray[5]
                            thisevent['sub_position'] = linearray[6]                     
                        elif ((thisyear == 1942) & (linearray[1] == 'trotb101')):
                            thisevent['sub_team'] = 1
                        elif ((thisyear == 1944) & (linearray[1] == 'pylee101')):
                            thisevent['sub_team'] = linearray[4]
                            thisevent['sub_batting_order'] = linearray[5]
                            thisevent['sub_position'] = linearray[6]
                        elif ((thisyear == 1949) & (linearray[1] == 'burgs101')):
                            thisevent['sub_team'] = linearray[4]
                            thisevent['sub_batting_order'] = linearray[5]
                            thisevent['sub_position'] = linearray[6]
                        elif ((thisyear == 1965) & (linearray[1] == 'scher101')):
                            thisevent['sub_team'] = linearray[4]
                            thisevent['sub_batting_order'] = linearray[5]
                            thisevent['sub_position'] = linearray[6]                        
                        
                elif (thisevent['event_type'] == 'play'):
                    thisevent['inning'] = linearray[1]
                    thisevent['visitor_or_home'] = linearray[2]
                    thisevent['batter_id'] = linearray[3]
                    thisevent['pitch_count'] = linearray[4]
                    thisevent['pitch_sequence'] = linearray[5]
                    thisevent['event'] = linearray[6]
 
                elif (thisevent['event_type'] == 'badj'):
                    ladj_games.append(thisgamedict['id'])
                    thisevent['adjusting_batter_id'] = linearray[0]
                    thisevent['adjusting_batter_hand'] = linearray[1]

                elif (linearray[0] == 'padj'):
                    padj_games.append(thisgamedict['id'])
                    thisevent['adjusting_pitcher_id'] = linearray[0]
                    thisevent['adjusting_pitcher_hand'] = linearray[1]

                elif (linearray[0] == 'ladj'):
                    ladj_games.append(thisgamedict['id'])
#                    print(linearray)
                    thisevent['visitor_or_home'] = linearray[1]
                    thisevent['actual_order_of_wrong_batter'] = linearray[2]
#                    pprint(thisevent)

                else:
                    print('event type is weird!')
                events.append(thisevent)
                eventnum = eventnum + 1                    
                
    # add data for the final game, which we missed above
    thisgamedict['info'] = info_dict
    thisgamedict['visiting_starters'] = vis_start_list
    thisgamedict['home_starters'] = home_start_list
    thisgamedict['events'] = events
    thisgamedict['data'] = data_list
    
    allgames.append(thisgamedict)

    print('{0:.0f}: processed {1:,.0f} games.'.format(thisyear,len(allgames)))

    outfilename = 'raw/events_raw_{:.0f}.json'.format(thisyear)
    with open(outfilename, 'w') as f:
        json.dump(allgames, f)
    print('File written:',outfilename,'!')
print('Done!')

In [ ]:
ldf = pandas.DataFrame(ladj_games)
bdf = pandas.DataFrame(badj_games)
pdf = pandas.DataFrame(padj_games)

ldf.columns = ['gameID']
bdf.columns = ['gameID']
pdf.columns = ['gameID']

pdf